In [1]:
# In this scenario we are going to grab the info from excel files.
'''
We need:
1. Prices in different currencies
2. List with Pusblisher names.
3. Clean Publisher Name
4. Check Suppliers, currencies, publishers, discount, shipping costs
5. Check Suppliers special discounts
6. Check restrictions
7. Get landed Cost
8. List of ISBN to ananlyse.

'''

'\nWe need:\n1. Prices in different currencies\n2. List with Pusblisher names.\n3. Clean Publisher Name\n4. Check Suppliers, currencies, publishers, discount, shipping costs\n5. Check Suppliers special discounts\n6. Check restrictions\n7. Get landed Cost\n8. List of ISBN to ananlyse.\n\n'

In [2]:
# 0. Import Libraries and credentials
import pandas as pd
import pyodbc
import numpy as np
import sqlCredentials as sql

In [3]:
# 0. Get databse connection
# Let's pick all the titles gwe can get from the pricing matrix.

prod_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=BookXCenterProduction;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

prouc_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=PROCUREMENTDB;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

In [4]:
#1. Prices at different currencies
#this time we are going to use all the titles
list_price_sql = """
SELECT *
FROM Isbn.ListPrice
"""

list_price = pd.read_sql(list_price_sql, prod_db)
list_price.head()

,Id,Isbn,Currency,Price,PriceSource,PriceLastRefreshed,Timestamp
0,1,9780008135904,AUD,12.99,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-06 15:03:40.816666
1,2,9780008135904,CND,9.99,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-06 15:03:40.816666
2,3,9780008135904,GBP,4.99,BertramsInventory,2020-04-06 13:00:00.0235684,2020-04-06 15:03:40.816666
3,4,9780008135904,NZD,14.99,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-06 15:03:40.816666
4,5,9780008135935,AUD,12.99,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-06 15:03:40.816666


In [5]:
list_price.shape

(1622645, 7)

In [6]:
list_price['PriceSource'].unique()

array(['BowkerNepModel', 'BertramsInventory', 'GoldenList',
       'BookDepotInventory', 'NielsenFtp', 'BowkerApiModel',
       'TitlePageScraper', 'PearsonCanadaScraperModel',
       'SAPPurchaseOrder', 'AbePolandScraper'], dtype=object)

In [7]:
list_price.drop(columns = ['Id', 'PriceSource', 'PriceLastRefreshed', 'Timestamp'], inplace = True)
list_price.head()

,Isbn,Currency,Price
0,9780008135904,AUD,12.99
1,9780008135904,CND,9.99
2,9780008135904,GBP,4.99
3,9780008135904,NZD,14.99
4,9780008135935,AUD,12.99


In [8]:
bilblo_sql = """
SELECT *
FROM Isbn.Bibliography
"""
biblio = pd.read_sql(bilblo_sql, prod_db)
biblio.head()

,Isbn,Title,TitleSource,TitleLastRefreshed,Publisher,PublisherSource,PublisherLastRefreshed,Author,AuthorSource,AuthorLastRefreshed,...,PublicationDateSource,PublicationDateLastRefreshed,Copyright,CopyrightSource,CopyrightLastRefreshed,Binding,BindingSource,BindingLastRefreshed,Timestamp,Edition
0,9780000002402,None,None,None,W KLUWER,GoldenList,2020-04-06 12:00:01.1700000,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-06 15:02:49.383,None
1,9780000002679,None,None,None,W KLUWER,GoldenList,2020-04-06 12:00:01.1700000,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-06 15:02:49.383,None
2,9780000003102,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-06 15:02:49.383,None
3,9780000003201,None,None,None,W KLUWER,GoldenList,2020-04-06 12:00:01.1700000,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-06 15:02:49.383,None
4,9780000006493,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-06 15:02:49.383,None


In [9]:
sup_list = pd.merge(list_price, biblio, how='left', on = 'Isbn' )

In [10]:
sup_list.head()

,Isbn,Currency,Price,Title,TitleSource,TitleLastRefreshed,Publisher,PublisherSource,PublisherLastRefreshed,Author,...,PublicationDateSource,PublicationDateLastRefreshed,Copyright,CopyrightSource,CopyrightLastRefreshed,Binding,BindingSource,BindingLastRefreshed,Timestamp,Edition
0,9780008135904,AUD,12.99,Collins Mandarin Chinese Phrasebook and Dictio...,BertramsInventory,2020-04-06 13:00:00.0235684,None,None,None,None,...,BowkerNepModel,2020-03-06 21:00:00.0081342,2017,BowkerNepModel,2020-03-06 21:00:00.0081342,Paperback,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-06 15:02:49.383,3
1,9780008135904,CND,9.99,Collins Mandarin Chinese Phrasebook and Dictio...,BertramsInventory,2020-04-06 13:00:00.0235684,None,None,None,None,...,BowkerNepModel,2020-03-06 21:00:00.0081342,2017,BowkerNepModel,2020-03-06 21:00:00.0081342,Paperback,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-06 15:02:49.383,3
2,9780008135904,GBP,4.99,Collins Mandarin Chinese Phrasebook and Dictio...,BertramsInventory,2020-04-06 13:00:00.0235684,None,None,None,None,...,BowkerNepModel,2020-03-06 21:00:00.0081342,2017,BowkerNepModel,2020-03-06 21:00:00.0081342,Paperback,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-06 15:02:49.383,3
3,9780008135904,NZD,14.99,Collins Mandarin Chinese Phrasebook and Dictio...,BertramsInventory,2020-04-06 13:00:00.0235684,None,None,None,None,...,BowkerNepModel,2020-03-06 21:00:00.0081342,2017,BowkerNepModel,2020-03-06 21:00:00.0081342,Paperback,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-06 15:02:49.383,3
4,9780008135935,AUD,12.99,Collins Portuguese Phrasebook and Dictionary G...,BertramsInventory,2020-04-06 13:00:00.0235684,None,None,None,None,...,BowkerNepModel,2020-03-06 21:00:00.0081342,2016,BowkerNepModel,2020-03-06 21:00:00.0081342,Paperback,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-06 15:02:49.383,4


In [11]:
sup_list.columns

Index(['Isbn', 'Currency', 'Price', 'Title', 'TitleSource',
       'TitleLastRefreshed', 'Publisher', 'PublisherSource',
       'PublisherLastRefreshed', 'Author', 'AuthorSource',
       'AuthorLastRefreshed', 'PublicationDate', 'PublicationDateSource',
       'PublicationDateLastRefreshed', 'Copyright', 'CopyrightSource',
       'CopyrightLastRefreshed', 'Binding', 'BindingSource',
       'BindingLastRefreshed', 'Timestamp', 'Edition'],
      dtype='object')

In [12]:
sup_list = sup_list[['Isbn','Currency', 'Price','Title', 'Publisher', 'Author']]

In [13]:
sup_list.head()

,Isbn,Currency,Price,Title,Publisher,Author
0,9780008135904,AUD,12.99,Collins Mandarin Chinese Phrasebook and Dictio...,None,None
1,9780008135904,CND,9.99,Collins Mandarin Chinese Phrasebook and Dictio...,None,None
2,9780008135904,GBP,4.99,Collins Mandarin Chinese Phrasebook and Dictio...,None,None
3,9780008135904,NZD,14.99,Collins Mandarin Chinese Phrasebook and Dictio...,None,None
4,9780008135935,AUD,12.99,Collins Portuguese Phrasebook and Dictionary G...,None,None


In [14]:
sup_list['Publisher'].unique()

array([None, 'HARPER COLLINS', 'HarperElement', ...,
       'African Books Collective', 'Mkuki na Nyota',
       'Sub-Saharan Publishers'], dtype=object)

In [15]:
sup_list = sup_list.fillna('N/A')
sup_list.isna().sum()

Isbn         0
Currency     0
Price        0
Title        0
Publisher    0
Author       0
dtype: int64

In [16]:
# 3. Clean publisher
dictio_sql = """
SELECT *
FROM dbo.PublisherDictionary
"""

dictio = pd.read_sql(dictio_sql, prouc_db)
dictio.head()

,PublisherLong,PublisherShort
0,"""Abrams, Inc.""",ABRAMS
1,"""Addison-Wesley Longman, Incorporated""",PEARSON
2,"""Addison-Wesley Longman, Limited""",PEARSON
3,"""Allyn & Bacon, Incorporated""",PEARSON
4,"""American Occupational Therapy Association, In...","American Occupational Therapy Association, Inc..."


In [17]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

In [18]:
for col in list(dictio.columns):
    dictio[col] = dictio.apply(lambda x: clean_up(x[col]), axis =1)

In [19]:
dictio.head()

,PublisherLong,PublisherShort
0,"""ABRAMS, INC.""",ABRAMS
1,"""ADDISON-WESLEY LONGMAN, INCORPORATED""",PEARSON
2,"""ADDISON-WESLEY LONGMAN, LIMITED""",PEARSON
3,"""ALLYN & BACON, INCORPORATED""",PEARSON
4,"""AMERICAN OCCUPATIONAL THERAPY ASSOCIATION, IN...","AMERICAN OCCUPATIONAL THERAPY ASSOCIATION, INC..."


In [20]:
for col in list(sup_list.drop(columns = 'Price').columns):
    sup_list[col] = sup_list.apply(lambda x: clean_up(x[col]), axis =1)

In [21]:
sup_list.head()

,Isbn,Currency,Price,Title,Publisher,Author
0,9780008135904,AUD,12.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A
1,9780008135904,CND,9.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A
2,9780008135904,GBP,4.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A
3,9780008135904,NZD,14.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A
4,9780008135935,AUD,12.99,COLLINS PORTUGUESE PHRASEBOOK AND DICTIONARY G...,N/A,N/A


In [22]:
# Merge on Publisher
sup_list2 = pd.merge(sup_list, dictio, how='left', left_on = 'Publisher', right_on = 'PublisherLong' )

In [23]:
sup_list2.head()

,Isbn,Currency,Price,Title,Publisher,Author,PublisherLong,PublisherShort
0,9780008135904,AUD,12.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A,N/A,N/A
1,9780008135904,CND,9.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A,N/A,N/A
2,9780008135904,GBP,4.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A,N/A,N/A
3,9780008135904,NZD,14.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A,N/A,N/A
4,9780008135935,AUD,12.99,COLLINS PORTUGUESE PHRASEBOOK AND DICTIONARY G...,N/A,N/A,N/A,N/A


In [24]:
sup_list2['Currency'].unique()

array(['AUD', 'CND', 'GBP', 'NZD', 'USD', 'EURO', 'PLN'], dtype=object)

In [25]:
# 4. get suppliers discount
sup_pub = pd.read_excel('../../../../Vendors/.  Vendors Details\supplier_procurement_details.xlsx',  sheet_name='pub')
sup_pub.head()

,Supplier,Publisher,Currency,Discount,Shipping Cost,max quantity,Min,Region
0,COINFO,3D TOTAL,AUD,0.22,3.3,10000,0,AUSTRALIA
1,COINFO,ABC CLIO,AUD,0.30,3.3,10000,0,AUSTRALIA
2,COINFO,AIAA,AUD,0.30,3.3,10000,0,AUSTRALIA
3,COINFO,ALIEN & UNWIN,AUD,0.33,3.3,10000,0,AUSTRALIA
4,COINFO,Amer Psych Ass.,AUD,0.33,3.3,10000,0,AUSTRALIA


In [26]:
for col in ['Supplier', 'Publisher', 'Currency', 'Region']:
    sup_pub[col] = sup_pub.apply(lambda x: clean_up(x[col]), axis =1)

In [27]:
sup_list3 = pd.merge(sup_list2, sup_pub, how='inner', left_on = ['Currency', 'PublisherShort'], right_on = ['Currency', 'Publisher'] )

In [28]:
sup_list3

,Isbn,Currency,Price,Title,Publisher_x,Author,PublisherLong,PublisherShort,Supplier,Publisher_y,Discount,Shipping Cost,max quantity,Min,Region
0,9780060466381,GBP,10.99,N/A,T&F,N/A,T&F,T&F,ABE,T&F,0.35,3.47,10000,0,EUROPE
1,9780060466398,GBP,9.99,N/A,T&F,N/A,T&F,T&F,ABE,T&F,0.35,3.47,10000,0,EUROPE
2,9780003020656,GBP,53.99,N/A,T&F,N/A,T&F,T&F,ABE,T&F,0.35,3.47,10000,0,EUROPE
3,9780013510758,GBP,90.00,N/A,T&F,N/A,T&F,T&F,ABE,T&F,0.35,3.47,10000,0,EUROPE
4,9780041900071,GBP,100.00,N/A,T&F,N/A,T&F,T&F,ABE,T&F,0.35,3.47,10000,0,EUROPE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464351,9789027232656,AUD,71.95,INTRODUCING SOCIOLINGUISTICS: <STRONG>SECOND E...,JOHN BENJAMINS PUBLISHING,RAJEND MESTHRIE,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA
464352,9789027241801,AUD,81.95,INCOMPLETE ACQUISITION IN BILINGUALISM: RE-EXA...,JOHN BENJAMINS PUBLISHING,SILVINA MONTRUL,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA
464353,9789027252722,AUD,78.95,"PIDGINS, CREOLES AND MIXED LANGUAGES: AN INTRO...",JOHN BENJAMINS PUBLISHING,VIVEKA VELUPILLAI,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA
464354,9789027219060,AUD,75.95,COGNITIVE EXPLORATION OF LANGUAGE AND LINGUISTICS,JOHN BENJAMINS PUBLISHING,RENE DIRVEN,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA


In [29]:
sup_list3.drop(columns = ['Publisher_x', 'PublisherLong', 'Publisher_y'], inplace = True)

In [30]:
sup_list3.head()

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region
0,9780060466381,GBP,10.99,N/A,N/A,T&F,ABE,0.35,3.47,10000,0,EUROPE
1,9780060466398,GBP,9.99,N/A,N/A,T&F,ABE,0.35,3.47,10000,0,EUROPE
2,9780003020656,GBP,53.99,N/A,N/A,T&F,ABE,0.35,3.47,10000,0,EUROPE
3,9780013510758,GBP,90.00,N/A,N/A,T&F,ABE,0.35,3.47,10000,0,EUROPE
4,9780041900071,GBP,100.00,N/A,N/A,T&F,ABE,0.35,3.47,10000,0,EUROPE


In [31]:
# 5. Check Suppliers special discounts
sp_disc = pd.read_excel("../../../../Vendors/. Short Discounted/all_short_discount_list.xlsx")

In [32]:
sp_disc.head()

,SUPPLIER,ISBN,DISCOUNT
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


In [33]:
sp_disc.dtypes

SUPPLIER     object
ISBN          int64
DISCOUNT    float64
dtype: object

In [34]:
sp_disc['ISBN'] = sp_disc['ISBN'].astype(str)
sp_disc.dtypes

SUPPLIER     object
ISBN         object
DISCOUNT    float64
dtype: object

In [35]:
for col in ['SUPPLIER', 'ISBN']:
    sp_disc[col] = sp_disc.apply(lambda x: clean_up(x[col]), axis =1)

sp_disc.head()

,SUPPLIER,ISBN,DISCOUNT
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


In [36]:
sup_list4 = pd.merge(sup_list3, sp_disc, how='left', left_on = ['Supplier', 'Isbn'], right_on = ['SUPPLIER', 'ISBN'] )

In [37]:
sup_list4.head()

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT
0,9780060466381,GBP,10.99,N/A,N/A,T&F,ABE,0.35,3.47,10000,0,EUROPE,NaN,NaN,NaN
1,9780060466398,GBP,9.99,N/A,N/A,T&F,ABE,0.35,3.47,10000,0,EUROPE,NaN,NaN,NaN
2,9780003020656,GBP,53.99,N/A,N/A,T&F,ABE,0.35,3.47,10000,0,EUROPE,NaN,NaN,NaN
3,9780013510758,GBP,90.00,N/A,N/A,T&F,ABE,0.35,3.47,10000,0,EUROPE,NaN,NaN,NaN
4,9780041900071,GBP,100.00,N/A,N/A,T&F,ABE,0.35,3.47,10000,0,EUROPE,NaN,NaN,NaN


In [38]:
sup_list4[sup_list4['SUPPLIER']=='ABE']

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT
158818,9781437734928,GBP,80.00,CORRECTIONS,JOHN T. WHITEHEAD; KIMBERLY D. DODSON; BRADLEY...,T&F,ABE,0.35,3.47,10000,0,EUROPE,ABE,9781437734928,0.35
187030,9780134782843,GBP,142.99,CHEZ NOUS: BRANCHE SUR LE MONDE FRANCOPHONE,MARY ELLEN SCULLEN,PEARSON,ABE,0.43,3.47,10000,0,EUROPE,ABE,9780134782843,0.35
198660,9780781766173,GBP,65.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781766173,0.35
198661,9780781768702,GBP,55.00,ACSM'S RESOURCES FOR CLINICAL EXERCISE PHYSIOLOGY,AMERICAN COLLEGE OF SPORTS MEDICINE,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781768702,0.35
198667,9780781774833,GBP,49.99,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781774833,0.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451130,9780393937282,GBP,15.00,DEFINING A NATION,AINSLIE T. EMBREE; MARK C. CARNES,NORTON,ABE,0.30,3.47,10000,0,EUROPE,ABE,9780393937282,-0.05
451167,9780393937275,GBP,17.99,CONFUCIANISM AND THE SUCCESSSION CRISIS OF THE...,DANIEL K. GARDNER; MARK C. CARNES,NORTON,ABE,0.30,3.47,10000,0,EUROPE,ABE,9780393937275,0.25
451180,9780393930047,GBP,49.99,PHYSICS FOR ENGINEERS AND SCIENTISTS,HANS C. OHANIAN; JOHN T. MARKERT,NORTON,ABE,0.30,3.47,10000,0,EUROPE,ABE,9780393930047,0.25
451411,9780393925227,GBP,53.00,ECONOMICS OF THE PUBLIC SECTOR,JOSEPH E. STIGLITZ; JAY K. ROSENGARD,NORTON,ABE,0.30,3.47,10000,0,EUROPE,ABE,9780393925227,0.25


In [39]:
sup_list4.isna().sum()

Isbn                   0
Currency               0
Price                  0
Title                  0
Author                 0
PublisherShort         0
Supplier               0
Discount               0
Shipping Cost          0
max quantity           0
Min                    0
Region                 0
SUPPLIER          462865
ISBN              462865
DISCOUNT          462865
dtype: int64

In [40]:
sup_list4['final_discount'] = np.where(sup_list4['DISCOUNT'].isna(), sup_list4['Discount'],sup_list4['DISCOUNT'])

In [41]:
sup_list4.head()

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT,final_discount
0,9780060466381,GBP,10.99,N/A,N/A,T&F,ABE,0.35,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.35
1,9780060466398,GBP,9.99,N/A,N/A,T&F,ABE,0.35,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.35
2,9780003020656,GBP,53.99,N/A,N/A,T&F,ABE,0.35,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.35
3,9780013510758,GBP,90.00,N/A,N/A,T&F,ABE,0.35,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.35
4,9780041900071,GBP,100.00,N/A,N/A,T&F,ABE,0.35,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.35


In [42]:
sup_list4[sup_list4['Isbn'] == '9780781762021']

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT,final_discount
300934,9780781762021,GBP,57.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781762021,0.35,0.35


In [43]:
sup_list4 = sup_list4[['Isbn','Title', 'Author', 'PublisherShort', 'Supplier','Currency', 'Price', 'final_discount' ,'Shipping Cost', 'max quantity', 'Min', 'Region']]

In [44]:
sup_list4.head()

,Isbn,Title,Author,PublisherShort,Supplier,Currency,Price,final_discount,Shipping Cost,max quantity,Min,Region
0,9780060466381,N/A,N/A,T&F,ABE,GBP,10.99,0.35,3.47,10000,0,EUROPE
1,9780060466398,N/A,N/A,T&F,ABE,GBP,9.99,0.35,3.47,10000,0,EUROPE
2,9780003020656,N/A,N/A,T&F,ABE,GBP,53.99,0.35,3.47,10000,0,EUROPE
3,9780013510758,N/A,N/A,T&F,ABE,GBP,90.00,0.35,3.47,10000,0,EUROPE
4,9780041900071,N/A,N/A,T&F,ABE,GBP,100.00,0.35,3.47,10000,0,EUROPE


In [45]:
sup_list4.columns = map(str.lower, sup_list4.columns)

In [46]:
sup_list4.head()

,isbn,title,author,publishershort,supplier,currency,price,final_discount,shipping cost,max quantity,min,region
0,9780060466381,N/A,N/A,T&F,ABE,GBP,10.99,0.35,3.47,10000,0,EUROPE
1,9780060466398,N/A,N/A,T&F,ABE,GBP,9.99,0.35,3.47,10000,0,EUROPE
2,9780003020656,N/A,N/A,T&F,ABE,GBP,53.99,0.35,3.47,10000,0,EUROPE
3,9780013510758,N/A,N/A,T&F,ABE,GBP,90.00,0.35,3.47,10000,0,EUROPE
4,9780041900071,N/A,N/A,T&F,ABE,GBP,100.00,0.35,3.47,10000,0,EUROPE


In [47]:
# Add western campus
# Shipping Cost = 2
# 0-150CAD 10% off
# 150-200CAD 12% off
# > 200CAD 14%

can_sup = sup_list2[sup_list2['Currency'] == 'CND']
can_sup.columns = map(str.lower, can_sup.columns)
can_sup['supplier'] = 'WC'
can_sup['final_discount'] = np.where(can_sup['price']<150, 0.1, np.where((can_sup['price'] >=150) & (can_sup['price']<200),0.12,0.14))
can_sup['max quantity'] = 10000
can_sup['region'] = 'NORTH AMERICA'
can_sup['shipping cost'] = 2
can_sup['min'] = 0
can_sup.head()

,isbn,currency,price,title,publisher,author,publisherlong,publishershort,supplier,final_discount,max quantity,region,shipping cost,min
1,9780008135904,CND,9.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0
5,9780008135935,CND,7.99,COLLINS PORTUGUESE PHRASEBOOK AND DICTIONARY G...,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0
13,9780008136628,CND,28.99,COLLINS WORLD ATLAS: ILLUSTRATED EDITION,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0
27,9780008141790,CND,17.99,COLLINS ENGLISH POCKET DICTIONARY AND THESAURUS,HARPER COLLINS,N/A,HARPER COLLINS,HARPER COLLINS,WC,0.1,10000,NORTH AMERICA,2,0
34,9780008141806,CND,17.99,COLLINS ENGLISH POCKET DICTIONARY,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0


In [48]:
can_sup = can_sup[['isbn', 'title', 'author', 'publishershort', 'supplier','currency','price','final_discount', 'shipping cost', 'max quantity', 'min', 'region']]
can_sup['currency'] = np.where(can_sup['currency'] == 'CND', 'CAD',0)

In [49]:
can_sup.head()

,isbn,title,author,publishershort,supplier,currency,price,final_discount,shipping cost,max quantity,min,region
1,9780008135904,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A,WC,CAD,9.99,0.1,2,10000,0,NORTH AMERICA
5,9780008135935,COLLINS PORTUGUESE PHRASEBOOK AND DICTIONARY G...,N/A,N/A,WC,CAD,7.99,0.1,2,10000,0,NORTH AMERICA
13,9780008136628,COLLINS WORLD ATLAS: ILLUSTRATED EDITION,N/A,N/A,WC,CAD,28.99,0.1,2,10000,0,NORTH AMERICA
27,9780008141790,COLLINS ENGLISH POCKET DICTIONARY AND THESAURUS,N/A,HARPER COLLINS,WC,CAD,17.99,0.1,2,10000,0,NORTH AMERICA
34,9780008141806,COLLINS ENGLISH POCKET DICTIONARY,N/A,N/A,WC,CAD,17.99,0.1,2,10000,0,NORTH AMERICA


In [50]:
sup_list5 = sup_list4.append(can_sup, ignore_index= True)
sup_list5.head()

,isbn,title,author,publishershort,supplier,currency,price,final_discount,shipping cost,max quantity,min,region
0,9780060466381,N/A,N/A,T&F,ABE,GBP,10.99,0.35,3.47,10000,0,EUROPE
1,9780060466398,N/A,N/A,T&F,ABE,GBP,9.99,0.35,3.47,10000,0,EUROPE
2,9780003020656,N/A,N/A,T&F,ABE,GBP,53.99,0.35,3.47,10000,0,EUROPE
3,9780013510758,N/A,N/A,T&F,ABE,GBP,90.00,0.35,3.47,10000,0,EUROPE
4,9780041900071,N/A,N/A,T&F,ABE,GBP,100.00,0.35,3.47,10000,0,EUROPE


In [51]:
sup_list5['currency'].unique()

array(['GBP', 'AUD', 'USD', 'CAD'], dtype=object)

In [52]:
# Exchange Rate
#We add the Exchange rate column
exchange_rate = """
SELECT *
FROM dbo.XChange
"""
exch_df = pd.read_sql(exchange_rate,prouc_db)


In [53]:
exch_df

,id,name,rate,updatedon
0,2,AUD,0.606715,2020-04-06 08:00:00.660
1,3,NZD,0.593201,2020-04-06 08:00:00.660
2,4,GBP,1.230762,2020-04-06 08:00:00.660
3,5,CAD,0.708642,2020-04-06 08:00:00.660
4,6,EUR,1.080013,2020-04-06 08:00:00.660
5,8,SGD,0.697284,2020-04-06 08:00:00.660
6,9,HKD,0.128982,2020-04-06 08:00:00.660
7,1,USD,1.000000,2020-04-06 08:00:00.660
8,10,TWD,0.033135,2020-04-06 08:00:00.660
9,11,CHF,1.022320,2020-04-06 08:00:00.660


In [54]:
sup_list6 = pd.merge(sup_list5, exch_df, how='left', left_on = 'currency', right_on = 'name' )
sup_list6['invoice_fee'] = np.where(sup_list6['supplier'] == 'COINFO', -1*sup_list6['price']*(1-sup_list6['disount'])  , 0)
sup_list6['lp_fee'] = 0
sup_list6.rename(columns={'publishershort': 'pub', 'final_discount': 'discount'}, inplace = True)


In [55]:
sup_list6 = sup_list6[['isbn', 'title', 'author', 'pub', 'supplier','currency','rate','price','discount','invoice_fee','lp_fee','shipping cost', 'max quantity', 'min', 'region']]
sup_list6.head()

,isbn,title,author,pub,supplier,currency,rate,price,discount,invoice_fee,lp_fee,shipping cost,max quantity,min,region
0,9780060466381,N/A,N/A,T&F,ABE,GBP,1.230762,10.99,0.35,0,0,3.47,10000,0,EUROPE
1,9780060466398,N/A,N/A,T&F,ABE,GBP,1.230762,9.99,0.35,0,0,3.47,10000,0,EUROPE
2,9780003020656,N/A,N/A,T&F,ABE,GBP,1.230762,53.99,0.35,0,0,3.47,10000,0,EUROPE
3,9780013510758,N/A,N/A,T&F,ABE,GBP,1.230762,90.00,0.35,0,0,3.47,10000,0,EUROPE
4,9780041900071,N/A,N/A,T&F,ABE,GBP,1.230762,100.00,0.35,0,0,3.47,10000,0,EUROPE


In [56]:
sup_list6.to_csv('suppliers_excel.csv', index= False)